In [9]:
import pandas as pd
import os
import sys
from pathlib import Path

def merge_and_sort_csvs(file_paths, output_filename=None, sort_column=None, duplicate_handling=None):
    """
    Merge multiple CSV files and sort by participant_id in ascending order.
    
    Parameters:
    file_paths (list): List of file paths to CSV files
    output_filename (str): Name for the output file
    sort_column (str): Column name to sort by
    duplicate_handling (str): How to handle duplicates
    
    Returns:
    pandas.DataFrame: The merged and sorted dataframe
    """
    
    # Use global defaults if not provided
    if output_filename is None:
        output_filename = OUTPUT_FILE
    if sort_column is None:
        sort_column = SORT_COLUMN
    if duplicate_handling is None:
        duplicate_handling = DUPLICATE_HANDLING
    
    # List to store dataframes
    dataframes = []
    
    # Read each CSV file
    for i, file_path in enumerate(file_paths, 1):
        try:
            if not os.path.exists(file_path):
                print(f"Error: File {file_path} not found")
                continue
                
            df = pd.read_csv(file_path)
            print(f"File {i}: Loaded {len(df)} rows from {file_path}")
            
            # Strip whitespace from headers
            df.columns = df.columns.str.strip()
            
            dataframes.append(df)
            
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            continue
    
    if not dataframes:
        print("No valid CSV files were loaded")
        return None
    
    # Merge all dataframes
    merged_df = pd.concat(dataframes, ignore_index=True)
    print(f"\nMerged data: {len(merged_df)} total rows")
    
    # Check for duplicates before processing
    if sort_column in merged_df.columns:
        duplicates = merged_df[merged_df.duplicated(subset=[sort_column], keep=False)]
        if len(duplicates) > 0:
            unique_duplicate_ids = duplicates[sort_column].unique()
            print(f"\nWarning: Found {len(duplicates)} rows with duplicate {sort_column} values")
            print(f"Duplicate {sort_column} values: {list(unique_duplicate_ids)}")
            
            # Handle duplicates based on configuration
            if duplicate_handling == 'keep_first':
                original_len = len(merged_df)
                merged_df = merged_df.drop_duplicates(subset=[sort_column], keep='first')
                print(f"Kept first occurrence of each duplicate. Removed {original_len - len(merged_df)} rows.")
            elif duplicate_handling == 'keep_last':
                original_len = len(merged_df)
                merged_df = merged_df.drop_duplicates(subset=[sort_column], keep='last')
                print(f"Kept last occurrence of each duplicate. Removed {original_len - len(merged_df)} rows.")
            elif duplicate_handling == 'keep_all':
                print("Keeping all duplicate rows as configured.")
            else:
                print(f"Unknown duplicate handling option: {duplicate_handling}. Keeping all rows.")
        else:
            print(f"No duplicate {sort_column} values found.")
    
    # Check if sort column exists
    if sort_column not in merged_df.columns:
        print(f"Warning: '{sort_column}' column not found in the data")
        print(f"Available columns: {list(merged_df.columns)}")
        return merged_df
    
    # Sort by sort_column in ascending order
    # Handle both numeric and string participant IDs
    try:
        # Try to convert to numeric first
        merged_df[f'{sort_column}_numeric'] = pd.to_numeric(merged_df[sort_column], errors='coerce')
        
        # If we have numeric values, sort by those, otherwise sort as strings
        if not merged_df[f'{sort_column}_numeric'].isna().all():
            merged_df = merged_df.sort_values(f'{sort_column}_numeric', na_position='last')
        else:
            merged_df = merged_df.sort_values(sort_column, key=lambda x: x.astype(str))
        
        # Drop the temporary numeric column
        if f'{sort_column}_numeric' in merged_df.columns:
            merged_df = merged_df.drop(f'{sort_column}_numeric', axis=1)
            
    except Exception as e:
        print(f"Error sorting by {sort_column}: {str(e)}")
        print("Sorting as strings instead...")
        merged_df = merged_df.sort_values(sort_column, key=lambda x: x.astype(str))
    
    print(f"Data sorted by {sort_column}")
    
    # Save to CSV
    merged_df.to_csv(output_filename, index=False)
    print(f"Saved merged and sorted data to: {output_filename}")
    
    return merged_df

# =====================================================
# CONFIGURATION VARIABLES - EDIT THESE
# =====================================================

# File paths for your CSV files
FILE1 = "/data/users2/nblair7/analysis_results/feedback_evaluation_scores1.csv"
FILE2 = "/data/users2/nblair7/analysis_results/SCOREST.csv" 
FILE3 = "/data/users2/nblair7/analysis_results/SCORES1.csv"
FILE4 = "/data/users2/nblair7/analysis_results/SCORES.csv"

# Output filename for merged data
OUTPUT_FILE = "/data/users2/nblair7/analysis_results/merged_sorted_data.csv"

# Sort column (should be 'participant_id' based on your headers)
SORT_COLUMN = "participant_id"

# Handle duplicate participant_ids
# Options: 'keep_all', 'keep_first', 'keep_last'
DUPLICATE_HANDLING = "keep_last"

# =====================================================
# END CONFIGURATION - Don't edit below unless needed
# =====================================================

def main():
    """
    Main function
    """
    
    # Use the configuration variables
    file_paths = [FILE1, FILE2, FILE3, FILE4]
    
    print("CSV File Merger and Sorter")
    print("=" * 40)
    print(f"Input files: {file_paths}")
    print(f"Output file: {OUTPUT_FILE}")
    print(f"Sort column: {SORT_COLUMN}")
    print(f"Duplicate handling: {DUPLICATE_HANDLING}")
    print("=" * 40)
    
    # Check if files exist
    existing_files = [f for f in file_paths if os.path.exists(f)]
    
    if len(existing_files) == 0:
        print("No CSV files found. Please check your file paths:")
        for f in file_paths:
            print(f"  - {f} {'✓' if os.path.exists(f) else '✗ (not found)'}")
        return
    
    if len(existing_files) < len(file_paths):
        print("Some files not found:")
        for f in file_paths:
            print(f"  - {f} {'✓' if os.path.exists(f) else '✗ (not found)'}")
        print(f"\nProceeding with {len(existing_files)} available files...")
    
    # Merge and sort the files
    result_df = merge_and_sort_csvs(existing_files)
    
    if result_df is not None:
        print(f"\nPreview of merged data (first 5 rows):")
        print("=" * 40)
        print(result_df.head())
        print(f"\nTotal rows: {len(result_df)}")
        print(f"Columns: {list(result_df.columns)}")
        
        # Show sort column range - FIXED: Use SORT_COLUMN instead of sort_column
        if SORT_COLUMN in result_df.columns:
            first_id = result_df[SORT_COLUMN].iloc[0]
            last_id = result_df[SORT_COLUMN].iloc[-1]
            print(f"{SORT_COLUMN} range: {first_id} to {last_id}")

if __name__ == "__main__":
    main()

CSV File Merger and Sorter
Input files: ['/data/users2/nblair7/analysis_results/feedback_evaluation_scores1.csv', '/data/users2/nblair7/analysis_results/SCOREST.csv', '/data/users2/nblair7/analysis_results/SCORES1.csv', '/data/users2/nblair7/analysis_results/SCORES.csv']
Output file: /data/users2/nblair7/analysis_results/merged_sorted_data.csv
Sort column: participant_id
Duplicate handling: keep_last
File 1: Loaded 20 rows from /data/users2/nblair7/analysis_results/feedback_evaluation_scores1.csv
File 2: Loaded 10 rows from /data/users2/nblair7/analysis_results/SCOREST.csv
File 3: Loaded 6 rows from /data/users2/nblair7/analysis_results/SCORES1.csv
File 4: Loaded 27 rows from /data/users2/nblair7/analysis_results/SCORES.csv

Merged data: 63 total rows

Duplicate participant_id values: [np.int64(380), np.int64(391), np.int64(393), np.int64(397), np.int64(402), np.int64(414), np.int64(415), np.int64(419), np.int64(425), np.int64(433)]
Kept last occurrence of each duplicate. Removed 12 ro

In [12]:
import pandas as pd
import numpy as np
import os

def smart_merge_csvs(primary_file, secondary_file, join_column="participant_id", 
                     columns_to_fill=None, output_file="smart_merged_data.csv"):
    """
    Smart merge that only fills missing values in specified columns from secondary file.
    
    Parameters:
    primary_file (str): Path to the main CSV file (this takes priority)
    secondary_file (str): Path to the file with data to fill missing values
    join_column (str): Column to join on (default: 'participant_id')
    columns_to_fill (list): List of columns to fill missing values for. If None, fills all common columns
    output_file (str): Output filename
    
    Returns:
    pandas.DataFrame: The merged dataframe
    """
    
    print("Smart CSV Merger - Fill Missing Values Only")
    print("=" * 50)
    
    # Read the files
    try:
        print(f"Reading primary file: {primary_file}")
        df_primary = pd.read_csv(primary_file)
        df_primary.columns = df_primary.columns.str.strip()
        print(f"  - Loaded {len(df_primary)} rows, {len(df_primary.columns)} columns")
        
        print(f"Reading secondary file: {secondary_file}")
        df_secondary = pd.read_csv(secondary_file)
        df_secondary.columns = df_secondary.columns.str.strip()
        print(f"  - Loaded {len(df_secondary)} rows, {len(df_secondary.columns)} columns")
        
    except Exception as e:
        print(f"Error reading files: {e}")
        return None
    
    # Check if join column exists in both files
    if join_column not in df_primary.columns:
        print(f"Error: '{join_column}' not found in primary file")
        print(f"Available columns in primary: {list(df_primary.columns)}")
        return None
    
    if join_column not in df_secondary.columns:
        print(f"Error: '{join_column}' not found in secondary file")
        print(f"Available columns in secondary: {list(df_secondary.columns)}")
        return None
    
    # Find common columns (excluding join column)
    common_columns = list(set(df_primary.columns) & set(df_secondary.columns))
    common_columns = [col for col in common_columns if col != join_column]
    
    print(f"\nCommon columns found: {common_columns}")
    
    # Determine which columns to fill
    if columns_to_fill is None:
        columns_to_fill = common_columns
        print(f"Will attempt to fill missing values in all common columns")
    else:
        # Validate that specified columns exist in both dataframes
        missing_in_primary = [col for col in columns_to_fill if col not in df_primary.columns]
        missing_in_secondary = [col for col in columns_to_fill if col not in df_secondary.columns]
        
        if missing_in_primary:
            print(f"Warning: These columns not found in primary file: {missing_in_primary}")
        if missing_in_secondary:
            print(f"Warning: These columns not found in secondary file: {missing_in_secondary}")
        
        # Keep only columns that exist in both
        columns_to_fill = [col for col in columns_to_fill 
                          if col in df_primary.columns and col in df_secondary.columns]
    
    print(f"Columns to fill: {columns_to_fill}")
    
    # Start with primary dataframe
    result_df = df_primary.copy()
    
    # Track changes for reporting
    changes_made = {}
    
    # For each column to fill
    for col in columns_to_fill:
        print(f"\nProcessing column '{col}':")
        
        # Count missing values in primary
        missing_mask = result_df[col].isna() | (result_df[col] == '') | (result_df[col] == 'NaN')
        missing_count = missing_mask.sum()
        print(f"  - Found {missing_count} missing values in primary file")
        
        if missing_count == 0:
            print(f"  - No missing values to fill in '{col}'")
            continue
        
        # Create a mapping from secondary file
        secondary_mapping = df_secondary.set_index(join_column)[col].to_dict()
        
        # Track successful fills
        fills_made = 0
        
        # Fill missing values
        for idx in result_df.index:
            if missing_mask.iloc[idx]:  # If this row has missing value
                participant_id = result_df.loc[idx, join_column]
                
                if participant_id in secondary_mapping:
                    secondary_value = secondary_mapping[participant_id]
                    
                    # Only fill if secondary value is not also missing
                    if pd.notna(secondary_value) and secondary_value != '' and secondary_value != 'NaN':
                        result_df.loc[idx, col] = secondary_value
                        fills_made += 1
        
        print(f"  - Successfully filled {fills_made} values")
        changes_made[col] = fills_made
    
    # Summary
    print(f"\n" + "=" * 50)
    print("SUMMARY OF CHANGES:")
    total_fills = sum(changes_made.values())
    print(f"Total values filled: {total_fills}")
    
    for col, count in changes_made.items():
        if count > 0:
            print(f"  - {col}: {count} values filled")
    
    # Save result
    result_df.to_csv(output_file, index=False)
    print(f"\nSaved merged data to: {output_file}")
    print(f"Final dataset: {len(result_df)} rows, {len(result_df.columns)} columns")
    
    return result_df

def preview_missing_data(file_path, join_column="participant_id"):
    """
    Preview which columns have missing data and how much
    """
    print(f"\nPREVIEW: Missing data analysis for {file_path}")
    print("-" * 60)
    
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()
        
        print(f"File: {file_path}")
        print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
        
        # Check for missing data in each column
        missing_info = []
        for col in df.columns:
            if col != join_column:  # Skip the join column
                missing_count = (df[col].isna() | (df[col] == '') | (df[col] == 'NaN')).sum()
                missing_pct = (missing_count / len(df)) * 100
                missing_info.append((col, missing_count, missing_pct))
        
        # Sort by missing count (descending)
        missing_info.sort(key=lambda x: x[1], reverse=True)
        
        print(f"\nMissing data by column:")
        for col, count, pct in missing_info:
            if count > 0:
                print(f"  {col}: {count} missing ({pct:.1f}%)")
        
        # Show columns with no missing data
        complete_columns = [col for col, count, pct in missing_info if count == 0]
        if complete_columns:
            print(f"\nColumns with complete data: {complete_columns}")
            
    except Exception as e:
        print(f"Error analyzing file: {e}")

# =====================================================
# CONFIGURATION - EDIT THESE SETTINGS
# =====================================================

# File paths
PRIMARY_FILE = "/data/users2/nblair7/analysis_results/merged_sorted_data.csv"  # This file takes priority
SECONDARY_FILE = "//data/users2/nblair7/analysis_results/eval_results_new.csv"  # Fill missing values from here

# Join column (usually participant_id)
JOIN_COLUMN = "participant_id"

# Specific columns to fill (set to None to fill all common columns)
# Example: COLUMNS_TO_FILL = ["score1", "score2", "feedback_rating"]
COLUMNS_TO_FILL = None  # Will fill all common columns

# Output file
OUTPUT_FILE = "/data/users2/nblair7/analysis_results/SCORES_FIXEDMERGE.csv"

# =====================================================
# MAIN EXECUTION
# =====================================================

def main():
    # First, let's preview the missing data in both files
    print("STEP 1: Analyzing missing data patterns...")
    preview_missing_data(PRIMARY_FILE, JOIN_COLUMN)
    preview_missing_data(SECONDARY_FILE, JOIN_COLUMN)
    
    print("\n" + "=" * 60)
    print("STEP 2: Performing smart merge...")
    
    # Perform the smart merge
    result = smart_merge_csvs(
        primary_file=PRIMARY_FILE,
        secondary_file=SECONDARY_FILE,
        join_column=JOIN_COLUMN,
        columns_to_fill=COLUMNS_TO_FILL,
        output_file=OUTPUT_FILE
    )
    
    if result is not None:
        print(f"\n" + "=" * 60)
        print("STEP 3: Final preview")
        print(f"First 5 rows of merged data:")
        print(result.head())
        
        print(f"\nColumns in final dataset: {list(result.columns)}")

if __name__ == "__main__":
    main()

STEP 1: Analyzing missing data patterns...

PREVIEW: Missing data analysis for /data/users2/nblair7/analysis_results/merged_sorted_data.csv
------------------------------------------------------------
File: /data/users2/nblair7/analysis_results/merged_sorted_data.csv
Rows: 51, Columns: 27

Missing data by column:
  iterations_required: 31 missing (60.8%)
  low_scores_detected: 31 missing (60.8%)
  final_coherence_score: 31 missing (60.8%)
  final_coherence_explanation: 31 missing (60.8%)
  initial_coherence_score: 31 missing (60.8%)
  initial_coherence_explanation: 31 missing (60.8%)
  final_completeness_score: 31 missing (60.8%)
  final_completeness_explanation: 31 missing (60.8%)
  initial_completeness_score: 31 missing (60.8%)
  initial_completeness_explanation: 31 missing (60.8%)
  final_specificity_score: 31 missing (60.8%)
  final_specificity_explanation: 31 missing (60.8%)
  initial_specificity_score: 31 missing (60.8%)
  initial_specificity_explanation: 31 missing (60.8%)
  fin

In [14]:
import pandas as pd
import numpy as np
import os

def smart_merge_csvs(primary_file, secondary_file, join_column="participant_id", 
                     columns_to_fill=None, output_file="smart_merged_data.csv"):
    """
    Smart merge that only fills missing values in specified columns from secondary file.
    
    Parameters:
    primary_file (str): Path to the main CSV file (this takes priority)
    secondary_file (str): Path to the file with data to fill missing values
    join_column (str): Column to join on (default: 'participant_id')
    columns_to_fill (list): List of columns to fill missing values for. If None, fills all common columns
    output_file (str): Output filename
    
    Returns:
    pandas.DataFrame: The merged dataframe
    """
    
    print("Smart CSV Merger - Fill Missing Values Only")
    print("=" * 50)
    
    # Read the files with better empty value handling
    try:
        print(f"Reading primary file: {primary_file}")
        df_primary = pd.read_csv(primary_file, na_values=['', ' ', 'nan', 'NaN', 'NULL', 'null'], keep_default_na=True)
        df_primary.columns = df_primary.columns.str.strip()
        print(f"  - Loaded {len(df_primary)} rows, {len(df_primary.columns)} columns")
        
        print(f"Reading secondary file: {secondary_file}")
        df_secondary = pd.read_csv(secondary_file, na_values=['', ' ', 'nan', 'NaN', 'NULL', 'null'], keep_default_na=True)
        df_secondary.columns = df_secondary.columns.str.strip()
        print(f"  - Loaded {len(df_secondary)} rows, {len(df_secondary.columns)} columns")
        
    except Exception as e:
        print(f"Error reading files: {e}")
        return None
    
    # Check if join column exists in both files
    if join_column not in df_primary.columns:
        print(f"Error: '{join_column}' not found in primary file")
        print(f"Available columns in primary: {list(df_primary.columns)}")
        return None
    
    if join_column not in df_secondary.columns:
        print(f"Error: '{join_column}' not found in secondary file")
        print(f"Available columns in secondary: {list(df_secondary.columns)}")
        return None
    
    # Find common columns (excluding join column)
    common_columns = list(set(df_primary.columns) & set(df_secondary.columns))
    common_columns = [col for col in common_columns if col != join_column]
    
    print(f"\nCommon columns found: {common_columns}")
    
    # Determine which columns to fill
    if columns_to_fill is None:
        columns_to_fill = common_columns
        print(f"Will attempt to fill missing values in all common columns")
    else:
        # Validate that specified columns exist in both dataframes
        missing_in_primary = [col for col in columns_to_fill if col not in df_primary.columns]
        missing_in_secondary = [col for col in columns_to_fill if col not in df_secondary.columns]
        
        if missing_in_primary:
            print(f"Warning: These columns not found in primary file: {missing_in_primary}")
        if missing_in_secondary:
            print(f"Warning: These columns not found in secondary file: {missing_in_secondary}")
        
        # Keep only columns that exist in both
        columns_to_fill = [col for col in columns_to_fill 
                          if col in df_primary.columns and col in df_secondary.columns]
    
    print(f"Columns to fill: {columns_to_fill}")
    
    # Start with primary dataframe
    result_df = df_primary.copy()
    
    # Track changes for reporting
    changes_made = {}
    
    # For each column to fill
    for col in columns_to_fill:
        print(f"\nProcessing column '{col}':")
        
        # Count missing values in primary - be more thorough
        missing_mask = (
            result_df[col].isna() | 
            (result_df[col] == '') | 
            (result_df[col] == 'NaN') |
            (result_df[col] == 'nan') |
            (result_df[col] == 'NULL') |
            (result_df[col] == 'null') |
            (result_df[col].astype(str).str.strip() == '')
        )
        missing_count = missing_mask.sum()
        print(f"  - Found {missing_count} missing values in primary file")
        
        if missing_count == 0:
            print(f"  - No missing values to fill in '{col}'")
            continue
        
        # Create a mapping from secondary file
        secondary_mapping = df_secondary.set_index(join_column)[col].to_dict()
        
        # Track successful fills
        fills_made = 0
        
        # Fill missing values
        for idx in result_df.index:
            if missing_mask.iloc[idx]:  # If this row has missing value
                participant_id = result_df.loc[idx, join_column]
                
                if participant_id in secondary_mapping:
                    secondary_value = secondary_mapping[participant_id]
                    
                    # Only fill if secondary value is not also missing - be more thorough
                    if (pd.notna(secondary_value) and 
                        secondary_value != '' and 
                        secondary_value != 'NaN' and
                        secondary_value != 'nan' and
                        secondary_value != 'NULL' and
                        secondary_value != 'null' and
                        str(secondary_value).strip() != ''):
                        result_df.loc[idx, col] = secondary_value
                        fills_made += 1
        
        print(f"  - Successfully filled {fills_made} values")
        changes_made[col] = fills_made
    
    # Summary
    print(f"\n" + "=" * 50)
    print("SUMMARY OF CHANGES:")
    total_fills = sum(changes_made.values())
    print(f"Total values filled: {total_fills}")
    
    for col, count in changes_made.items():
        if count > 0:
            print(f"  - {col}: {count} values filled")
    
    # Save result
    result_df.to_csv(output_file, index=False)
    print(f"\nSaved merged data to: {output_file}")
    print(f"Final dataset: {len(result_df)} rows, {len(result_df.columns)} columns")
    
    return result_df

def debug_specific_row(primary_file, secondary_file, participant_id, join_column="participant_id"):
    """
    Debug a specific participant to see exactly what's happening
    """
    print(f"\nDEBUG: Analyzing participant {participant_id}")
    print("-" * 50)
    
    try:
        # Read files
        df_primary = pd.read_csv(primary_file, na_values=['', ' ', 'nan', 'NaN', 'NULL', 'null'], keep_default_na=True)
        df_primary.columns = df_primary.columns.str.strip()
        
        df_secondary = pd.read_csv(secondary_file, na_values=['', ' ', 'nan', 'NaN', 'NULL', 'null'], keep_default_na=True)
        df_secondary.columns = df_secondary.columns.str.strip()
        
        # Find the participant in both files
        primary_row = df_primary[df_primary[join_column] == participant_id]
        secondary_row = df_secondary[df_secondary[join_column] == participant_id]
        
        if len(primary_row) == 0:
            print(f"Participant {participant_id} not found in primary file")
            return
        if len(secondary_row) == 0:
            print(f"Participant {participant_id} not found in secondary file")
            return
        
        primary_row = primary_row.iloc[0]
        secondary_row = secondary_row.iloc[0]
        
        # Find common columns
        common_columns = list(set(df_primary.columns) & set(df_secondary.columns))
        common_columns = [col for col in common_columns if col != join_column]
        
        print(f"Common columns: {common_columns}")
        print(f"\nColumn-by-column comparison:")
        
        for col in common_columns:
            primary_val = primary_row[col]
            secondary_val = secondary_row[col]
            
            # Check if primary is missing
            is_primary_missing = (
                pd.isna(primary_val) or
                primary_val == '' or
                primary_val == 'NaN' or
                primary_val == 'nan' or
                primary_val == 'NULL' or
                primary_val == 'null' or
                str(primary_val).strip() == ''
            )
            
            print(f"  {col}:")
            print(f"    Primary: '{primary_val}' (missing: {is_primary_missing})")
            print(f"    Secondary: '{secondary_val}'")
            print(f"    Would fill: {is_primary_missing and pd.notna(secondary_val) and str(secondary_val).strip() != ''}")
            print()
            
    except Exception as e:
        print(f"Error in debug: {e}")

def preview_missing_data(file_path, join_column="participant_id"):
    """
    Preview which columns have missing data and how much
    """
    print(f"\nPREVIEW: Missing data analysis for {file_path}")
    print("-" * 60)
    
    try:
        df = pd.read_csv(file_path, na_values=['', ' ', 'nan', 'NaN', 'NULL', 'null'], keep_default_na=True)
        df.columns = df.columns.str.strip()
        
        print(f"File: {file_path}")
        print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
        
        # Check for missing data in each column
        missing_info = []
        for col in df.columns:
            if col != join_column:  # Skip the join column
                missing_count = (
                    df[col].isna() | 
                    (df[col] == '') | 
                    (df[col] == 'NaN') |
                    (df[col] == 'nan') |
                    (df[col] == 'NULL') |
                    (df[col] == 'null') |
                    (df[col].astype(str).str.strip() == '')
                ).sum()
                missing_pct = (missing_count / len(df)) * 100
                missing_info.append((col, missing_count, missing_pct))
        
        # Sort by missing count (descending)
        missing_info.sort(key=lambda x: x[1], reverse=True)
        
        print(f"\nMissing data by column:")
        for col, count, pct in missing_info:
            if count > 0:
                print(f"  {col}: {count} missing ({pct:.1f}%)")
        
        # Show columns with no missing data
        complete_columns = [col for col, count, pct in missing_info if count == 0]
        if complete_columns:
            print(f"\nColumns with complete data: {complete_columns}")
            
    except Exception as e:
        print(f"Error analyzing file: {e}")

# =====================================================
# CONFIGURATION - EDIT THESE SETTINGS
# =====================================================

# File paths
PRIMARY_FILE = "/data/users2/nblair7/analysis_results/merged_sorted_data.csv"  # This file takes priority
SECONDARY_FILE = "/data/users2/nblair7/analysis_results/eval_results_new.csv"  # Fill missing values from here

# Join column (usually participant_id)
JOIN_COLUMN = "participant_id"

# Specific columns to fill (set to None to fill all common columns)
# Example: COLUMNS_TO_FILL = ["score1", "score2", "feedback_rating"]
COLUMNS_TO_FILL = None  # Will fill all common columns

# Output file
OUTPUT_FILE = "/data/users2/nblair7/analysis_results/MERGESDSCORES.csv"

# =====================================================
# MAIN EXECUTION
# =====================================================

def main():
    # First, let's preview the missing data in both files
    print("STEP 1: Analyzing missing data patterns...")
    preview_missing_data(PRIMARY_FILE, JOIN_COLUMN)
    preview_missing_data(SECONDARY_FILE, JOIN_COLUMN)
    
    # Debug a specific participant (change this to your problem participant)
    print("\nSTEP 1.5: Debugging specific participant...")
    debug_specific_row(PRIMARY_FILE, SECONDARY_FILE, 402, JOIN_COLUMN)  # Change 402 to your participant ID
    
    print("\n" + "=" * 60)
    print("STEP 2: Performing smart merge...")
    
    # Perform the smart merge
    result = smart_merge_csvs(
        primary_file=PRIMARY_FILE,
        secondary_file=SECONDARY_FILE,
        join_column=JOIN_COLUMN,
        columns_to_fill=COLUMNS_TO_FILL,
        output_file=OUTPUT_FILE
    )
    
    if result is not None:
        print(f"\n" + "=" * 60)
        print("STEP 3: Final preview")
        print(f"First 5 rows of merged data:")
        print(result.head())
        
        print(f"\nColumns in final dataset: {list(result.columns)}")
        
        # Show a specific participant after merge
        if 402 in result[JOIN_COLUMN].values:
            print(f"\nParticipant 402 after merge:")
            participant_402 = result[result[JOIN_COLUMN] == 402].iloc[0]
            for col in result.columns:
                print(f"  {col}: '{participant_402[col]}'")

if __name__ == "__main__":
    main()

STEP 1: Analyzing missing data patterns...

PREVIEW: Missing data analysis for /data/users2/nblair7/analysis_results/merged_sorted_data.csv
------------------------------------------------------------
File: /data/users2/nblair7/analysis_results/merged_sorted_data.csv
Rows: 51, Columns: 27

Missing data by column:
  iterations_required: 31 missing (60.8%)
  low_scores_detected: 31 missing (60.8%)
  final_coherence_score: 31 missing (60.8%)
  final_coherence_explanation: 31 missing (60.8%)
  initial_coherence_score: 31 missing (60.8%)
  initial_coherence_explanation: 31 missing (60.8%)
  final_completeness_score: 31 missing (60.8%)
  final_completeness_explanation: 31 missing (60.8%)
  initial_completeness_score: 31 missing (60.8%)
  initial_completeness_explanation: 31 missing (60.8%)
  final_specificity_score: 31 missing (60.8%)
  final_specificity_explanation: 31 missing (60.8%)
  initial_specificity_score: 31 missing (60.8%)
  initial_specificity_explanation: 31 missing (60.8%)
  fin

In [15]:
import pandas as pd
import os

def reorder_csv_columns(input_file, output_file, desired_columns):
    """
    Reorder and select specific columns from a CSV file.
    
    Parameters:
    input_file (str): Path to the input CSV file
    output_file (str): Path for the output CSV file
    desired_columns (list): List of column names in the desired order
    
    Returns:
    pandas.DataFrame: The reordered dataframe
    """
    
    print("CSV Column Reorder Tool")
    print("=" * 50)
    
    try:
        # Read the input file
        print(f"Reading input file: {input_file}")
        df = pd.read_csv(input_file)
        df.columns = df.columns.str.strip()  # Remove any whitespace from column names
        
        print(f"  - Loaded {len(df)} rows, {len(df.columns)} columns")
        print(f"  - Original columns: {list(df.columns)}")
        
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    # Check which desired columns exist in the dataframe
    existing_columns = []
    missing_columns = []
    
    for col in desired_columns:
        if col in df.columns:
            existing_columns.append(col)
        else:
            missing_columns.append(col)
    
    print(f"\n📊 Column Analysis:")
    print(f"  ✅ Found {len(existing_columns)} of {len(desired_columns)} desired columns")
    print(f"  ❌ Missing {len(missing_columns)} columns")
    
    if existing_columns:
        print(f"\n✅ Columns that will be included:")
        for col in existing_columns:
            print(f"    - {col}")
    
    if missing_columns:
        print(f"\n❌ Missing columns (will be skipped):")
        for col in missing_columns:
            print(f"    - {col}")
    
    # Create new dataframe with only the existing desired columns
    if not existing_columns:
        print("❌ No desired columns found in the input file!")
        return None
    
    # Select and reorder columns
    result_df = df[existing_columns].copy()
    
    print(f"\n📋 Creating output file with {len(existing_columns)} columns:")
    print(f"  - New column order: {list(result_df.columns)}")
    
    # Save to output file
    result_df.to_csv(output_file, index=False)
    print(f"✅ Saved reordered data to: {output_file}")
    
    # Show summary
    print(f"\n📈 Summary:")
    print(f"  - Input rows: {len(df)}")
    print(f"  - Output rows: {len(result_df)}")
    print(f"  - Input columns: {len(df.columns)}")
    print(f"  - Output columns: {len(result_df.columns)}")
    
    return result_df

def preview_data(df, title, n_rows=3):
    """Preview sample data from a dataframe"""
    print(f"\n📋 {title}")
    print("-" * 60)
    
    if df is None or len(df) == 0:
        print("No data to display")
        return
    
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Show first few rows with truncated values for readability
    print(f"\nFirst {min(n_rows, len(df))} rows:")
    for i in range(min(n_rows, len(df))):
        print(f"\nRow {i+1}:")
        for col in df.columns:
            val = df.iloc[i][col]
            if pd.isna(val) or val == '':
                print(f"  {col}: [EMPTY]")
            else:
                val_str = str(val)
                # Truncate long values for display
                if len(val_str) > 80:
                    val_str = val_str[:77] + "..."
                print(f"  {col}: {val_str}")

# =====================================================
# CONFIGURATION - EDIT THESE SETTINGS
# =====================================================

# Input file path
INPUT_FILE = "/data/users2/nblair7/analysis_results/MERGESDSCORES.csv"  # Change this to your input file

# Output file path
OUTPUT_FILE = "/data/users2/nblair7/analysis_results/MERGEDS.csv"

# Desired columns in the exact order you want them
DESIRED_COLUMNS = [
    "participant_id",
    "coherence",
    "coherence_explanation", 
    "completeness",
    "completeness_explanation",
    "specificity", 
    "specificity_explanation",
    "accuracy",
    "accuracy_explanation"
]

# Alternative: If you want to try the "final_" versions as backup, uncomment this:
# BACKUP_COLUMN_MAPPING = {
#     "coherence": ["coherence", "final_coherence_score"],
#     "coherence_explanation": ["coherence_explanation", "final_coherence_explanation"],
#     "completeness": ["completeness", "final_completeness_score"],
#     "completeness_explanation": ["completeness_explanation", "final_completeness_explanation"],
#     "specificity": ["specificity", "final_specificity_score"],
#     "specificity_explanation": ["specificity_explanation", "final_specificity_explanation"],
#     "accuracy": ["accuracy", "final_accuracy_score"],
#     "accuracy_explanation": ["accuracy_explanation", "final_accuracy_explanation"]
# }

# =====================================================
# MAIN EXECUTION
# =====================================================

def main():
    print("CSV Column Reorder and Selection Tool")
    print("=" * 60)
    
    print(f"Input file: {INPUT_FILE}")
    print(f"Output file: {OUTPUT_FILE}")
    print(f"Desired columns: {DESIRED_COLUMNS}")
    
    # Check if input file exists
    if not os.path.exists(INPUT_FILE):
        print(f"❌ Error: Input file '{INPUT_FILE}' not found!")
        print("Please check the file path and try again.")
        return
    
    # Show original data structure
    try:
        original_df = pd.read_csv(INPUT_FILE)
        original_df.columns = original_df.columns.str.strip()
        preview_data(original_df, "ORIGINAL DATA STRUCTURE", 2)
    except Exception as e:
        print(f"Error previewing original data: {e}")
        return
    
    print(f"\n" + "=" * 60)
    print("PROCESSING...")
    
    # Reorder the columns
    result_df = reorder_csv_columns(INPUT_FILE, OUTPUT_FILE, DESIRED_COLUMNS)
    
    if result_df is not None:
        print(f"\n" + "=" * 60)
        print("RESULTS:")
        preview_data(result_df, "FINAL REORDERED DATA", 3)
        
        print(f"\n✅ SUCCESS!")
        print(f"Your reordered CSV has been saved as: {OUTPUT_FILE}")
        print(f"The file now has {len(result_df.columns)} columns in your desired order.")
        
        # Show the header that was created
        print(f"\n📋 New header line:")
        print(",".join(result_df.columns))
    else:
        print(f"\n❌ FAILED to create reordered file.")

if __name__ == "__main__":
    main()

CSV Column Reorder and Selection Tool
Input file: /data/users2/nblair7/analysis_results/MERGESDSCORES.csv
Output file: /data/users2/nblair7/analysis_results/MERGEDS.csv
Desired columns: ['participant_id', 'coherence', 'coherence_explanation', 'completeness', 'completeness_explanation', 'specificity', 'specificity_explanation', 'accuracy', 'accuracy_explanation']

📋 ORIGINAL DATA STRUCTURE
------------------------------------------------------------
Shape: (51, 27)
Columns: ['participant_id', 'iterations_required', 'low_scores_detected', 'final_coherence_score', 'final_coherence_explanation', 'initial_coherence_score', 'initial_coherence_explanation', 'final_completeness_score', 'final_completeness_explanation', 'initial_completeness_score', 'initial_completeness_explanation', 'final_specificity_score', 'final_specificity_explanation', 'initial_specificity_score', 'initial_specificity_explanation', 'final_accuracy_score', 'final_accuracy_explanation', 'initial_accuracy_score', 'initial_

In [17]:
import pandas as pd
import os

def append_csvs_with_replacement(file1, file2, join_column="participant_id", output_file="combined_data.csv"):
    """
    Append two CSV files where file2 data replaces any matching rows from file1.
    
    Parameters:
    file1 (str): Path to the first CSV file (base data)
    file2 (str): Path to the second CSV file (replacement/additional data)
    join_column (str): Column to match on for replacements (default: 'participant_id')
    output_file (str): Path for the output CSV file
    
    Returns:
    pandas.DataFrame: The combined dataframe
    """
    
    print("CSV Append with Replacement Tool")
    print("=" * 50)
    
    # Read both files
    try:
        print(f"Reading File 1 (base): {file1}")
        df1 = pd.read_csv(file1)
        df1.columns = df1.columns.str.strip()
        print(f"  - Loaded {len(df1)} rows, {len(df1.columns)} columns")
        
        print(f"Reading File 2 (replacement/additional): {file2}")
        df2 = pd.read_csv(file2)
        df2.columns = df2.columns.str.strip()
        print(f"  - Loaded {len(df2)} rows, {len(df2.columns)} columns")
        
    except Exception as e:
        print(f"Error reading files: {e}")
        return None
    
    # Check if join column exists in both files
    if join_column not in df1.columns:
        print(f"Error: '{join_column}' not found in File 1")
        print(f"Available columns in File 1: {list(df1.columns)}")
        return None
    
    if join_column not in df2.columns:
        print(f"Error: '{join_column}' not found in File 2")
        print(f"Available columns in File 2: {list(df2.columns)}")
        return None
    
    # Show column comparison
    print(f"\n📊 Column Analysis:")
    file1_cols = set(df1.columns)
    file2_cols = set(df2.columns)
    
    common_cols = file1_cols & file2_cols
    file1_only = file1_cols - file2_cols
    file2_only = file2_cols - file1_cols
    
    print(f"  Common columns: {len(common_cols)}")
    if common_cols:
        print(f"    {list(common_cols)}")
    
    if file1_only:
        print(f"  File 1 only: {len(file1_only)}")
        print(f"    {list(file1_only)}")
    
    if file2_only:
        print(f"  File 2 only: {len(file2_only)}")
        print(f"    {list(file2_only)}")
    
    # Find overlapping participants
    participants_file1 = set(df1[join_column].values)
    participants_file2 = set(df2[join_column].values)
    
    overlapping = participants_file1 & participants_file2
    file1_only_participants = participants_file1 - participants_file2
    file2_only_participants = participants_file2 - participants_file1
    
    print(f"\n👥 Participant Analysis:")
    print(f"  File 1 participants: {len(participants_file1)}")
    print(f"  File 2 participants: {len(participants_file2)}")
    print(f"  Overlapping (will be replaced): {len(overlapping)}")
    print(f"  File 1 only (will be kept): {len(file1_only_participants)}")
    print(f"  File 2 only (will be added): {len(file2_only_participants)}")
    
    if overlapping:
        print(f"  Overlapping participant IDs: {sorted(list(overlapping))}")
    
    # Step 1: Remove overlapping participants from file1
    print(f"\n🔄 Processing...")
    df1_filtered = df1[~df1[join_column].isin(participants_file2)].copy()
    print(f"  Step 1: Removed {len(df1) - len(df1_filtered)} overlapping rows from File 1")
    
    # Step 2: Combine the dataframes
    # First ensure both dataframes have the same columns
    all_columns = sorted(list(file1_cols | file2_cols))
    print(f"  Step 2: Standardizing columns - total columns in result: {len(all_columns)}")
    
    # Add missing columns with NaN values
    for col in all_columns:
        if col not in df1_filtered.columns:
            df1_filtered[col] = pd.NA
        if col not in df2.columns:
            df2[col] = pd.NA
    
    # Reorder columns to match
    df1_filtered = df1_filtered[all_columns]
    df2 = df2[all_columns]
    
    # Step 3: Concatenate
    result_df = pd.concat([df1_filtered, df2], ignore_index=True)
    print(f"  Step 3: Combined dataframes - final result: {len(result_df)} rows")
    
    # Step 4: Sort by participant_id if it's numeric
    try:
        result_df[f'{join_column}_numeric'] = pd.to_numeric(result_df[join_column], errors='coerce')
        if not result_df[f'{join_column}_numeric'].isna().all():
            result_df = result_df.sort_values(f'{join_column}_numeric', na_position='last')
            print(f"  Step 4: Sorted by {join_column} (numeric)")
        else:
            result_df = result_df.sort_values(join_column, key=lambda x: x.astype(str))
            print(f"  Step 4: Sorted by {join_column} (alphabetic)")
        
        # Drop the temporary numeric column
        if f'{join_column}_numeric' in result_df.columns:
            result_df = result_df.drop(f'{join_column}_numeric', axis=1)
    except:
        result_df = result_df.sort_values(join_column, key=lambda x: x.astype(str))
        print(f"  Step 4: Sorted by {join_column}")
    
    # Save result
    result_df.to_csv(output_file, index=False)
    print(f"\n✅ Saved combined data to: {output_file}")
    
    # Final summary
    print(f"\n📋 Final Summary:")
    print(f"  Total rows: {len(result_df)}")
    print(f"  Total columns: {len(result_df.columns)}")
    print(f"  Replacements made: {len(overlapping)}")
    print(f"  New participants added: {len(file2_only_participants)}")
    print(f"  Participants kept from File 1: {len(file1_only_participants)}")
    
    return result_df

def show_sample_replacements(file1, file2, join_column="participant_id", n_samples=3):
    """
    Show examples of what replacements will be made
    """
    print(f"\n🔍 SAMPLE REPLACEMENTS (showing up to {n_samples} examples)")
    print("-" * 60)
    
    try:
        df1 = pd.read_csv(file1)
        df1.columns = df1.columns.str.strip()
        df2 = pd.read_csv(file2)
        df2.columns = df2.columns.str.strip()
        
        # Find overlapping participants
        participants_file1 = set(df1[join_column].values)
        participants_file2 = set(df2[join_column].values)
        overlapping = list(participants_file1 & participants_file2)
        
        if not overlapping:
            print("No overlapping participants found - this will be a simple append.")
            return
        
        # Show a few examples
        for i, participant_id in enumerate(overlapping[:n_samples]):
            print(f"\nExample {i+1}: Participant {participant_id}")
            
            # Get data from both files
            row1 = df1[df1[join_column] == participant_id].iloc[0]
            row2 = df2[df2[join_column] == participant_id].iloc[0]
            
            # Find common columns
            common_cols = list(set(df1.columns) & set(df2.columns))
            common_cols = [col for col in common_cols if col != join_column]
            
            print(f"  Will replace File 1 data with File 2 data:")
            for col in common_cols[:5]:  # Show first 5 columns
                val1 = row1[col] if col in df1.columns else "[Missing]"
                val2 = row2[col] if col in df2.columns else "[Missing]"
                
                # Truncate long values
                if pd.notna(val1) and len(str(val1)) > 40:
                    val1 = str(val1)[:37] + "..."
                if pd.notna(val2) and len(str(val2)) > 40:
                    val2 = str(val2)[:37] + "..."
                
                print(f"    {col}:")
                print(f"      OLD: {val1}")
                print(f"      NEW: {val2}")
            
            if len(common_cols) > 5:
                print(f"    ... and {len(common_cols) - 5} more columns")
    
    except Exception as e:
        print(f"Error showing sample replacements: {e}")

# =====================================================
# CONFIGURATION - EDIT THESE SETTINGS
# =====================================================

# File paths
FILE2 = "/data/users2/nblair7/analysis_results/eval_results_new.csv"      # Base file (data will be kept unless replaced)
FILE1 = "/data/users2/nblair7/analysis_results/SCORES_R.csv"         # Replacement/additional file (this data takes priority)

# Join column for matching rows
JOIN_COLUMN = "participant_id"

# Output file
OUTPUT_FILE = "/data/users2/nblair7/analysis_results/newfindatascores.csv"

# =====================================================
# MAIN EXECUTION
# =====================================================

def main():
    print("CSV Append with Replacement Tool")
    print("=" * 60)
    
    print(f"File 1 (base): {FILE1}")
    print(f"File 2 (replacement/additional): {FILE2}")
    print(f"Join column: {JOIN_COLUMN}")
    print(f"Output file: {OUTPUT_FILE}")
    
    # Check if files exist
    if not os.path.exists(FILE1):
        print(f"❌ Error: File 1 '{FILE1}' not found!")
        return
    
    if not os.path.exists(FILE2):
        print(f"❌ Error: File 2 '{FILE2}' not found!")
        return
    
    # Show what replacements will be made
    show_sample_replacements(FILE1, FILE2, JOIN_COLUMN)
    
    print(f"\n" + "=" * 60)
    print("PROCESSING...")
    
    # Perform the append with replacement
    result = append_csvs_with_replacement(FILE1, FILE2, JOIN_COLUMN, OUTPUT_FILE)
    
    if result is not None:
        print(f"\n" + "=" * 60)
        print("✅ SUCCESS!")
        
        # Show sample of final data
        print(f"\nSample of final combined data:")
        print(f"First 3 rows:")
        for i in range(min(3, len(result))):
            participant_id = result.iloc[i][JOIN_COLUMN]
            print(f"  Row {i+1}: Participant {participant_id}")
            
        print(f"\nFinal file saved as: {OUTPUT_FILE}")
        print(f"Total participants in final file: {len(result)}")

if __name__ == "__main__":
    main()

CSV Append with Replacement Tool
File 1 (base): /data/users2/nblair7/analysis_results/SCORES_R.csv
File 2 (replacement/additional): /data/users2/nblair7/analysis_results/eval_results_new.csv
Join column: participant_id
Output file: /data/users2/nblair7/analysis_results/newfindatascores.csv

🔍 SAMPLE REPLACEMENTS (showing up to 3 examples)
------------------------------------------------------------

Example 1: Participant 385
  Will replace File 1 data with File 2 data:
    specificity_explanation:
      OLD: Score: 2
Explanation: The assessment ...
      NEW: Score: 4
Explanation: While the asses...
    completeness:
      OLD: 1.0
      NEW: 4
    accuracy:
      OLD: 2.0
      NEW: 3
    completeness_explanation:
      OLD: Score: 1
Explanation: This assessment...
      NEW: Score: 4
Explanation: The assessment ...
    specificity:
      OLD: 2.0
      NEW: 4
    ... and 3 more columns

Example 2: Participant 386
  Will replace File 1 data with File 2 data:
    specificity_explanati

In [18]:
import pandas as pd
import os

def append_csvs_with_replacement(file1, file2, join_column="participant_id", output_file="combined_data.csv"):
    """
    Append two CSV files where file2 data replaces any matching rows from file1.
    """
    
    print("CSV Append with Replacement Tool")
    print("=" * 50)
    
    # Read both files
    try:
        print(f"Reading File 1 (base): {file1}")
        df1 = pd.read_csv(file1)
        df1.columns = df1.columns.str.strip()
        print(f"  - Loaded {len(df1)} rows, {len(df1.columns)} columns")
        
        print(f"Reading File 2 (replacement/additional): {file2}")
        df2 = pd.read_csv(file2)
        df2.columns = df2.columns.str.strip()
        print(f"  - Loaded {len(df2)} rows, {len(df2.columns)} columns")
        
    except Exception as e:
        print(f"Error reading files: {e}")
        return None
    
    # Check if join column exists in both files
    if join_column not in df1.columns:
        print(f"Error: '{join_column}' not found in File 1")
        print(f"Available columns in File 1: {list(df1.columns)}")
        return None
    
    if join_column not in df2.columns:
        print(f"Error: '{join_column}' not found in File 2")
        print(f"Available columns in File 2: {list(df2.columns)}")
        return None
    
    # Show column comparison
    print(f"\n📊 Column Analysis:")
    file1_cols = set(df1.columns)
    file2_cols = set(df2.columns)
    
    common_cols = file1_cols & file2_cols
    file1_only = file1_cols - file2_cols
    file2_only = file2_cols - file1_cols
    
    print(f"  Common columns: {len(common_cols)}")
    if common_cols:
        print(f"    {list(common_cols)}")
    
    if file1_only:
        print(f"  File 1 only: {len(file1_only)}")
        print(f"    {list(file1_only)}")
    
    if file2_only:
        print(f"  File 2 only: {len(file2_only)}")
        print(f"    {list(file2_only)}")
    
    # Find overlapping participants
    participants_file1 = set(df1[join_column].values)
    participants_file2 = set(df2[join_column].values)
    
    overlapping = participants_file1 & participants_file2
    file1_only_participants = participants_file1 - participants_file2
    file2_only_participants = participants_file2 - participants_file1
    
    print(f"\n👥 Participant Analysis:")
    print(f"  File 1 participants: {len(participants_file1)}")
    print(f"  File 2 participants: {len(participants_file2)}")
    print(f"  Overlapping (will be replaced): {len(overlapping)}")
    print(f"  File 1 only (will be kept): {len(file1_only_participants)}")
    print(f"  File 2 only (will be added): {len(file2_only_participants)}")
    
    if overlapping:
        print(f"  Overlapping participant IDs: {sorted(list(overlapping))}")
    
    # Step 1: Remove overlapping participants from file1
    print(f"\n🔄 Processing...")
    df1_filtered = df1[~df1[join_column].isin(participants_file2)].copy()
    print(f"  Step 1: Removed {len(df1) - len(df1_filtered)} overlapping rows from File 1")
    
    # Step 2: Standardize columns - ensure participant_id comes first
    all_columns = [join_column] + sorted([col for col in (file1_cols | file2_cols) if col != join_column])
    print(f"  Step 2: Standardizing columns - total columns in result: {len(all_columns)}")
    
    # Add missing columns with NaN values
    for col in all_columns:
        if col not in df1_filtered.columns:
            df1_filtered[col] = pd.NA
        if col not in df2.columns:
            df2[col] = pd.NA
    
    # Reorder columns
    df1_filtered = df1_filtered[all_columns]
    df2 = df2[all_columns]
    
    # Step 3: Concatenate
    result_df = pd.concat([df1_filtered, df2], ignore_index=True)
    print(f"  Step 3: Combined dataframes - final result: {len(result_df)} rows")
    
    # Step 4: Sort by participant_id if it's numeric
    try:
        result_df[f'{join_column}_numeric'] = pd.to_numeric(result_df[join_column], errors='coerce')
        if not result_df[f'{join_column}_numeric'].isna().all():
            result_df = result_df.sort_values(f'{join_column}_numeric', na_position='last')
            print(f"  Step 4: Sorted by {join_column} (numeric)")
        else:
            result_df = result_df.sort_values(join_column, key=lambda x: x.astype(str))
            print(f"  Step 4: Sorted by {join_column} (alphabetic)")
        
        # Drop the temporary numeric column
        if f'{join_column}_numeric' in result_df.columns:
            result_df = result_df.drop(f'{join_column}_numeric', axis=1)
    except:
        result_df = result_df.sort_values(join_column, key=lambda x: x.astype(str))
        print(f"  Step 4: Sorted by {join_column}")
    
    # Save result
    result_df.to_csv(output_file, index=False)
    print(f"\n✅ Saved combined data to: {output_file}")
    
    # Final summary
    print(f"\n📋 Final Summary:")
    print(f"  Total rows: {len(result_df)}")
    print(f"  Total columns: {len(result_df.columns)}")
    print(f"  Replacements made: {len(overlapping)}")
    print(f"  New participants added: {len(file2_only_participants)}")
    print(f"  Participants kept from File 1: {len(file1_only_participants)}")
    
    return result_df
